In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Cargar archivo Excel
file_path = '/mnt/data/Internet.xlsx'
excel_data = pd.ExcelFile(file_path)

# Función para limpieza de datos y EDA por hoja
def clean_and_explore(excel_data):
    for sheet_name in excel_data.sheet_names:
        print(f"\n--- Análisis de la hoja: {sheet_name} ---")
        df = excel_data.parse(sheet_name)

        # Cantidad de valores faltantes antes de la limpieza
        missing_values_before = df.isnull().sum().sum()
        print(f"Cantidad de valores faltantes antes de la limpieza: {missing_values_before}")

        # Eliminación de duplicados
        duplicates_before = df.duplicated().sum()
        print(f"Cantidad de registros duplicados antes de la limpieza: {duplicates_before}")
        df = df.drop_duplicates()

        # Limpieza de datos: eliminar filas con datos faltantes
        df_cleaned = df.dropna()

        # Cantidad de valores eliminados
        missing_values_after = df.isnull().sum().sum()
        print(f"Cantidad de valores faltantes eliminados: {missing_values_before - missing_values_after}")

        # Conversión de tipos: numéricos y cadenas de texto
        for column in df_cleaned.columns:
            if df_cleaned[column].dtype == np.object:
                df_cleaned[column] = df_cleaned[column].astype(str)
            elif pd.api.types.is_numeric_dtype(df_cleaned[column]):
                df_cleaned[column] = pd.to_numeric(df_cleaned[column], errors='coerce')

        # Identificación de valores atípicos (outliers) usando el método IQR
        numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            Q1 = df_cleaned[col].quantile(0.25)
            Q3 = df_cleaned[col].quantile(0.75)
            IQR = Q3 - Q1
            outliers = df_cleaned[(df_cleaned[col] < (Q1 - 1.5 * IQR)) | (df_cleaned[col] > (Q3 + 1.5 * IQR))]
            print(f"Cantidad de valores atípicos en la columna '{col}': {len(outliers)}")

            # Boxplot para visualizar outliers
            plt.figure(figsize=(8, 4))
            sns.boxplot(x=df_cleaned[col])
            plt.title(f"Boxplot para la columna {col}")
            plt.show()

        # Mostrar información básica
        print("\nResumen estadístico:\n", df_cleaned.describe(include='all'))
        print("\nTipos de datos:\n", df_cleaned.dtypes)

        # Análisis visual: distribuciones y correlaciones
        sns.pairplot(df_cleaned.select_dtypes(include=[np.number]))
        plt.suptitle(f"Distribución de variables numéricas en {sheet_name}", y=1.02)
        plt.show()

        # Correlación de variables numéricas
        plt.figure(figsize=(10, 8))
        sns.heatmap(df_cleaned.corr(), annot=True, cmap='coolwarm')
        plt.title(f"Mapa de calor de correlaciones para {sheet_name}")
        plt.show()

clean_and_explore(excel_data)

